# 翻车的人不配有睡眠（误


今天(2019-06-26)，居然想当然地把 `list_ += [1, ]` 等价于 `list_.append([1,])`了。。

为了不再翻车。。。整理一下python内置对象中 **\_\_iadd__**  的功能。

## too long dont read

int:
- 支持7个赋值运算符(`+=`、`-=`、`*=`、`/=`、`%=`、`**=`、`//=`)

str:
- 支持`+=`，`+=`的作用和`+`类似，不是原地操作
- 不支持`-=`、`/=`
- 对于`*=`，只支持 `Str *= Int`，作用是拼接n个字符串，不是原地操作

list:
- 支持`List += List`, 原地操作 ！！
- 不支持`-=`
- 对于`*=`，只支持 `List *= Int`，作用是拼接n个列表，原地操作！！
- 不支持`/=`

tuple:
- 支持`+=`，`+=`的作用和`+`类似，不是原地操作！！
- 不支持`-=`、`/=`
- 对于`*=`，只支持 `Tuple *= Int`，作用是拼接n个字符串，不是原地操作

set:
- 不支持`+=`、`*=`、`/=`
- 支持`Set -= Set`！！原地操作！！

dict:
- 不支持`+=`、`-=`、`*=`、`/=`

## detail

In [192]:
def show_obj_detail(obj):
    print(type(obj))
    print(id(obj))
    print(obj)
    print('-'*5)

## int

In [193]:
a = 1
show_obj_detail(a)
b = 1
show_obj_detail(b)
a += b
show_obj_detail(a)

<class 'int'>
99163476751136
1
-----
<class 'int'>
99163476751136
1
-----
<class 'int'>
99163476751168
2
-----


In [194]:
a = 1
show_obj_detail(a)
b = 2
show_obj_detail(b)
a = a+b
show_obj_detail(a)

<class 'int'>
99163476751136
1
-----
<class 'int'>
99163476751168
2
-----
<class 'int'>
99163476751200
3
-----


从输出可以看到，`int` 类型中，`+=` 和 `+` 的功能挺一致的，都是重新分配地址。

有意思的是，`a = 1` 和 `b = 1`的时候，a 和 b是使用了同一个地址，
稍微测试了下，在python3.7中,给多个变量赋值，在 -5 ~ 256 区间,id一样的，python有一个策略是常用int指向同一个地址，达到节省内存的目的

In [195]:
a = 1
a += 2
print(a)
a = 1
a  -= 2
print(a)
a = 1
a *= 3
print(a)
a = 1
a /= 2
print(a)
a = 3
a %= 2
print(a)
a = 3
a **= 2
print(a)
a = 5 
a //= 2
print(a)


3
-1
3
0.5
1
9
2


由于上面代码正常运行，说明int类型对于7个赋值运算符都支持。

In [196]:

a = 11
b = 10
a &= b
print(a)

a = 11
b = 10
a |= b
print(a)

a = 11
b = 10
a ^= b
print(a)

a = 1111
b = 2
a <<= b
print(a)

a = 60
b = 2
a >>= b
print(a)

10
11
1
4444
15


看～正常运行，说明int类型支持(`&`, `|`, `^`, `<<`, `>>`)五个位运算符和等号的组合

如果是 `a ~= b`的话，会报`SyntaxError: invalid syntax`

报错说明不支持 `~` 和 `=`的组合，这个很好理解，毕竟`~`是一元运算符

*注： 这里SyntaxError是不会被try except捕捉到的，有兴趣可以去试试～原因是SyntaxError是在**代码解析的时候**就已经抛出，而try生效的地方是在**代码运行的时候***

In [197]:
a = 1
b = 2

print(a >= b)
print(a <= b)


False
True


`<=` 和 `>=` 就不属于赋值预算符号了，只是比较运算符。

## str

In [198]:
a = 'a'
show_obj_detail(a)
b = 'b'
show_obj_detail(b)
a += b
show_obj_detail(a)
a = a + b
show_obj_detail(a)


<class 'str'>
140085249393080
a
-----
<class 'str'>
140085249680472
b
-----
<class 'str'>
140084945454280
ab
-----
<class 'str'>
140084945454224
abb
-----


str类型支持`+=` ,并且 `+=` 的行为和`+`类似，值和地址都会更新

In [199]:
a = 'a'
show_obj_detail(a)
b = 'b'
show_obj_detail(b)


try:
    a -= b
except TypeError as e:
    print(e)
else:
    show_obj_detail()


<class 'str'>
140085249393080
a
-----
<class 'str'>
140085249680472
b
-----
unsupported operand type(s) for -=: 'str' and 'str'


str类型不支持 `-=` 

In [200]:
a = 'a'
show_obj_detail(a)
b = 'b'
show_obj_detail(b)


try:
    a *= b
except TypeError as e:
    print(e)
else:
    show_obj_detail()





<class 'str'>
140085249393080
a
-----
<class 'str'>
140085249680472
b
-----
can't multiply sequence by non-int of type 'str'


str类型不支持 `*=` str


In [201]:
a = 'ab'
show_obj_detail(a)
a *= 2
print('a *= 2 --> ')
show_obj_detail(a)


<class 'str'>
140085170378992
ab
-----
a *= 2 --> 
<class 'str'>
140084945453832
abab
-----


但是str类型支持 `*=` int，作用是 拼凑 n个str，组成一个新的str。

In [202]:
a = 'ab'
show_obj_detail(a)

try:
    a /= 2
except TypeError as e:
    print(e)
    
try:
    a /= 'b'
except TypeError as e:
    print(e)

<class 'str'>
140085170378992
ab
-----
unsupported operand type(s) for /=: 'str' and 'int'
unsupported operand type(s) for /=: 'str' and 'str'


毫无疑问的是:
- unsupported operand type(s) for /=: 'str' and 'int'
- unsupported operand type(s) for /=: 'str' and 'str'

## list


In [203]:
a = [1]
show_obj_detail(a)
b = [2]
show_obj_detail(b)

a += b
show_obj_detail(a)

c = [1]
d = [2]
show_obj_detail(c+d)


<class 'list'>
140084945103624
[1]
-----
<class 'list'>
140084944810184
[2]
-----
<class 'list'>
140084945103624
[1, 2]
-----
<class 'list'>
140084944895112
[1, 2]
-----


注意，list类型中
- 如果是`a += b`这样的句式，会将b的数据更新到a中，a的id不变
- 如果是'a+b'这样的句式，会使用指针保存a+b的地址

In [204]:
a = [1]
show_obj_detail(a)
b = [2]
show_obj_detail(b)

try:
    a -= b
except TypeError as e:
    print(e)
else:
    show_obj_detail(a)

<class 'list'>
140084944895368
[1]
-----
<class 'list'>
140084945103624
[2]
-----
unsupported operand type(s) for -=: 'list' and 'list'


In [205]:
a = [1]
show_obj_detail(a)
b = [2]
show_obj_detail(b)

try:
    a *= b
except TypeError as e:
    print(e)
    
a *= 3
show_obj_detail(a)



<class 'list'>
140084944895432
[1]
-----
<class 'list'>
140084944895368
[2]
-----
can't multiply sequence by non-int of type 'list'
<class 'list'>
140084944895432
[1, 1, 1]
-----


对于乘法
- `List *= List`是**非法的**
- `List *= Int`是合法的(和str**一样**)
- `List *= Int`的指针在调用前后不变(和str**不一样**)

In [206]:
a = [1]
show_obj_detail(a)
b = [2]
show_obj_detail(b)

try:
    a /= b
except TypeError as e:
    print(e)
else:
    show_obj_detail(a)
    
try:
    a /= 3
except TypeError as e:
    print(e)
else:
    show_obj_detail(a)

<class 'list'>
140084944610824
[1]
-----
<class 'list'>
140084944895432
[2]
-----
unsupported operand type(s) for /=: 'list' and 'list'
unsupported operand type(s) for /=: 'list' and 'int'


显然，list不支持除法，无论除数是list还是int类型

## tuple

In [207]:
a = (1,)
show_obj_detail(a)
b = (2,)
show_obj_detail(b)

a += b
show_obj_detail(a)
    
c = a + b
show_obj_detail(c)

<class 'tuple'>
140084945144352
(1,)
-----
<class 'tuple'>
140084944864592
(2,)
-----
<class 'tuple'>
140084945184648
(1, 2)
-----
<class 'tuple'>
140084945417704
(1, 2, 2)
-----


- tuple支持`+=`
- 无论是`+=`还是`+`，都不是原地修改


In [208]:
a = (1,)
show_obj_detail(a)
b = (2,)
show_obj_detail(b)

try:
    a -= b
except TypeError as e:
    print(e)
else:
    show_obj_detail(a)

try:
    c = a - b
except TypeError as e:
    print(e)
else:
    show_obj_detail(c)

<class 'tuple'>
140084945315488
(1,)
-----
<class 'tuple'>
140084944822344
(2,)
-----
unsupported operand type(s) for -=: 'tuple' and 'tuple'
unsupported operand type(s) for -: 'tuple' and 'tuple'


In [209]:
a = (1,)
show_obj_detail(a)
b = (2,)
show_obj_detail(b)

try:
    a *= b
except TypeError as e:
    print(e)
else:
    show_obj_detail(a)
    
a *= 3
show_obj_detail(a)

show_obj_detail(a*3)

<class 'tuple'>
140084945316048
(1,)
-----
<class 'tuple'>
140084945276152
(2,)
-----
can't multiply sequence by non-int of type 'tuple'
<class 'tuple'>
140084945663104
(1, 1, 1)
-----
<class 'tuple'>
140085085711360
(1, 1, 1, 1, 1, 1, 1, 1, 1)
-----


- tuple只支持和**int类型**相乘,其他类型都🙅
- `Tuple *= Int` 和 `Tuple * Int`表现一样，都**不是原地修改**

In [210]:
a = (1,)
show_obj_detail(a)
b = (2,)
show_obj_detail(b)

try:
    _ = a / b
except TypeError as e:
    print(e)
try:
    a /=  b
except TypeError as e:
    print(e)
    


<class 'tuple'>
140084945313864
(1,)
-----
<class 'tuple'>
140084945432760
(2,)
-----
unsupported operand type(s) for /: 'tuple' and 'tuple'
unsupported operand type(s) for /=: 'tuple' and 'tuple'


## set

In [211]:
a = {'a'}
show_obj_detail(a)
b = {'a', 'b'}
show_obj_detail(b)

try:
    a += b
except TypeError as e:
    print(e)
    
try:
    a += 2
except TypeError as e:
    print(e) 
    
try:
    _ = a + b
except TypeError as e:
    print(e) 



<class 'set'>
140084945506088
{'a'}
-----
<class 'set'>
140084945503624
{'a', 'b'}
-----
unsupported operand type(s) for +=: 'set' and 'set'
unsupported operand type(s) for +=: 'set' and 'int'
unsupported operand type(s) for +: 'set' and 'set'


set类型不支持加法

In [212]:
a = {'a'}
show_obj_detail(a)
b = {'a', 'b'}
show_obj_detail(b)

try:
    a -= b
except TypeError as e:
    print(e)
else:
    print('a -= b')
    show_obj_detail(a)
    
a = {'a'}
show_obj_detail(a)
b = {'a', 'b'}
show_obj_detail(b)
try:
    tmp = a -  b
except TypeError as e:
    print(e)
else:
    print('a - b')
    show_obj_detail(tmp)


<class 'set'>
140084945504072
{'a'}
-----
<class 'set'>
140084945506088
{'a', 'b'}
-----
a -= b
<class 'set'>
140084945504072
set()
-----
<class 'set'>
140084945504296
{'a'}
-----
<class 'set'>
140084945504072
{'a', 'b'}
-----
a - b
<class 'set'>
140084945506088
set()
-----


然而，set类型支持减法！！
- `Set -= Set`的话，是原地减。（指针不变）
- `Set - Set`的话，是不是原地减。（指针变了）


In [213]:
a = {'a'}
show_obj_detail(a)
b = {'a', 'b'}
show_obj_detail(b)

try:
    a *= b
except TypeError as e:
    print(e)
else:
    print('a *= b')
    show_obj_detail(a)
    
try:
    a /= b
except TypeError as e:
    print(e)
else:
    print('a /= b')
    show_obj_detail(a)

<class 'set'>
140084945503624
{'a'}
-----
<class 'set'>
140084945504296
{'a', 'b'}
-----
unsupported operand type(s) for *=: 'set' and 'set'
unsupported operand type(s) for /=: 'set' and 'set'


## dict

In [214]:
a = {'a': 1}
show_obj_detail(a)
b = {'b': 2}
show_obj_detail(b)

try:
    a += b
except TypeError as e:
    print(e)
    
try:
    _ = a + b
except TypeError as e:
    print(e)


<class 'dict'>
140084945280240
{'a': 1}
-----
<class 'dict'>
140084945848576
{'b': 2}
-----
unsupported operand type(s) for +=: 'dict' and 'dict'
unsupported operand type(s) for +: 'dict' and 'dict'


In [215]:
a = {'a': 1}
show_obj_detail(a)
b = {'b': 2}
show_obj_detail(b)

try:
    a -= b
except TypeError as e:
    print(e)
    
try:
    _ = a - b
except TypeError as e:
    print(e)

<class 'dict'>
140084945037208
{'a': 1}
-----
<class 'dict'>
140084945280240
{'b': 2}
-----
unsupported operand type(s) for -=: 'dict' and 'dict'
unsupported operand type(s) for -: 'dict' and 'dict'


In [216]:
a = {'a': 1}
show_obj_detail(a)
b = {'b': 2}
show_obj_detail(b)

try:
    a *= b
except TypeError as e:
    print(e)
    
try:
    _ = a * b
except TypeError as e:
    print(e)

<class 'dict'>
140084945474112
{'a': 1}
-----
<class 'dict'>
140084945037208
{'b': 2}
-----
unsupported operand type(s) for *=: 'dict' and 'dict'
unsupported operand type(s) for *: 'dict' and 'dict'


In [217]:
a = {'a': 1}
show_obj_detail(a)
b = {'b': 2}
show_obj_detail(b)

try:
    a /= b
except TypeError as e:
    print(e)
    
try:
    _ = a / b
except TypeError as e:
    print(e)

<class 'dict'>
140084945023912
{'a': 1}
-----
<class 'dict'>
140084945474112
{'b': 2}
-----
unsupported operand type(s) for /=: 'dict' and 'dict'
unsupported operand type(s) for /: 'dict' and 'dict'
